In [2]:
import numpy as np
import pandas as pd

In [3]:
from game import *
from network_tp import *

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
def encodePred(label_pred):
    indMax = np.argmax(label_pred)
    if indMax == 0:
        return np.array([1.,0.,0.])
    elif indMax == 1:
        return np.array([0.,1.,0.])
    else:
        return np.array([0.,0.,1.])

def encodeLabel(label):
    if label == 1.0:
        label = np.array([1.,0.,0.])
    elif label == 0.0:
        label = np.array([0.,1.,0.])
    else:
        label = np.array([0.,0.,1.])
    return label

def score(label_pred,turn):
    indMax = np.argmax(label_pred)
    value = 0.
    if indMax == 0:
        if turn == 1:
            value = -label_pred[0]
        else:
            value = label_pred[0]
    elif indMax == 1:
        value = 0.
    else:
        if turn == 1:
            value = -label_pred[2]
        else:
            value = label_pred[2]
    return value

def encodeBoard(board):
    board = np.array(board)
    board = np.array([board[0:7,5],board[0:7,4],board[0:7,3],board[0:7,2],board[0:7,1],board[0:7,0]],dtype = float).flatten()
    for i in range(len(board)):
        if board[i] == 2.:
            board[i] = -1.
    return board

def playMove(neural_network,turn):
    values = np.zeros(7)
    for col in range(7):
        if game.board_at(col,5) == 0: # if the network can play in col
            # copy state of the game
            gameCopy = game.copy_state()
            # play the move
            gameCopy.place(col)
            # evaluate the resulting board
            input = encodeBoard(gameCopy.get_board())
            #print("input")
            #print(input)
            label_pred = neural_network.forward(input)
            #print("for col",col)
            #print(label_pred)
            values[col] = score(label_pred,turn)
            #print(input)
            #print(values[col])
        else:
            values[col] = -1
    
    #print(values)
    move = np.argmax(values)
    game.place(move)

def createData(dataset,lenTest):
    np.random.shuffle(dataset)
    x_train = np.array([dataset[0,0:42]])
    y_train = np.array([encodeLabel(dataset[0,42])])
    for i in range(1,dataset.shape[0]-lenTest):
        input = dataset[i,0:42]
        label = encodeLabel(dataset[i,42])
        x_train = np.vstack((x_train,input))
        y_train = np.vstack((y_train,label))

    x_test = np.array([dataset[dataset.shape[0]-lenTest,0:42]])
    y_test = np.array([encodeLabel(dataset[dataset.shape[0]-lenTest,42])])

    for i in range((dataset.shape[0]+1-lenTest),(dataset.shape[0])):
        input = dataset[i,0:42]
        label = encodeLabel(dataset[i,42])
        x_test = np.vstack((x_test,input))
        y_test = np.vstack((y_test,label))
    
    return x_train,y_train,x_test,y_test

In [5]:
def playGameAgainstHimself(neural_network):
    game.reset_game()

    playMove(neural_network,game.get_turn())
    playMove(neural_network,game.get_turn())

    board = encodeBoard(game.get_board())

    trainingSet = np.array([board])

    while game.get_win() is None:
        playMove(neural_network,game.get_turn())

        if game.get_win() is None:
            playMove(neural_network,game.get_turn())

        board = encodeBoard(game.get_board())

        trainingSet = np.vstack((trainingSet,board))

    winner = game.get_win()
    if winner == 2:
        winner = -1
    
    trainingSet = np.column_stack((trainingSet,[winner]*trainingSet.shape[0]))
    
    game.reset_game()

    return trainingSet

In [6]:
def createTrainingReinforcement(neural_network,nbGames=100):
    globalTrainingSet = playGameAgainstHimself(neural_network)
    for i in range(1,nbGames):
        trainingSet = playGameAgainstHimself(neural_network)
        globalTrainingSet = np.vstack((globalTrainingSet,trainingSet))
    return globalTrainingSet

In [18]:
def reinforcementTraining(neural_network,nbEpochs=100,nbGames=100):
    print("[INFO] Reinforcement...")
    lenTrainReinforcement = 50
    for i in range(nbEpochs):
        trainingSet = createTrainingReinforcement(neural_network,nbGames)
        shuffler = np.random.permutation(trainingSet.shape[0])
        trainingSet = trainingSet[shuffler]
        x_train,y_train,x_test,y_test = createData(trainingSet,lenTrainReinforcement)
        
        neural_network.train_sgd_perso(x_train,y_train,batch_size=50)
        
        goodPred = 0
        preds = 0

        for j in range(lenTrainReinforcement):
            label_pred = encodePred(neural_network.forward(x_test[j]))
            if (label_pred == y_test[j]).all():
                goodPred += 1
            preds += 1

        if i%10 == 0:
            print(j," - classification rate =",goodPred/preds)
        

In [7]:
print("[INFO] Loading file network_tp...")

dataset = pd.read_csv("data/c4_game_database.csv")

dataset = dataset.to_numpy()

dataset = dataset[0:50000,]

[INFO] Loading file network_tp...


In [8]:
print("[INFO] Create datasets...")

lenTrain = 10000

x_train,y_train,x_test,y_test = createData(dataset,lenTrain)

[INFO] Create datasets...


In [9]:
print("[INFO] Create network...")

nw_c4 = NeuralNetwork([42,16,16,16,3],0.1)

[INFO] Create network...


In [13]:
print("[INFO] Train...")

EPOCH = 50

for j in range(EPOCH+1):
    shuffler = np.random.permutation(x_train.shape[0])
    x_train = x_train[shuffler]
    y_train = y_train[shuffler]
    nw_c4.train_sgd_perso(x_train,y_train,batch_size=1000)

    goodPred = 0
    preds = 0

    for i in range(lenTrain):
        label_pred = encodePred(nw_c4.forward(x_test[i]))
        if (label_pred == y_test[i]).all():
            goodPred += 1
        preds += 1

    if j%10 == 0:
        print(j," - classification rate =",goodPred/preds)

[INFO] Train...
0  - classification rate = 0.5927
10  - classification rate = 0.6071
20  - classification rate = 0.6224
30  - classification rate = 0.6356
40  - classification rate = 0.6431
50  - classification rate = 0.6489


In [1]:
reinforcementTraining(nw_c4,nbEpochs=5,nbGames=100)

NameError: name 'reinforcementTraining' is not defined

In [17]:
def trainNetwork(network,trainingSet,epoch=50):
    for j in range(epoch+1):
        shuffler = np.random.permutation(trainingSet.shape[0])
        x_train = x_train[shuffler]
        y_train = y_train[shuffler]
        nw_c4.train_sgd_perso(x_train,y_train,batch_size=1000)

        goodPred = 0
        preds = 0

        for i in range(lenTrain):
            label_pred = encodePred(nw_c4.forward(x_test[i]))
            if (label_pred == y_test[i]).all():
                goodPred += 1
            preds += 1

        if j%10 == 0:
            print(j," - classification rate =",goodPred/preds)

In [21]:
view = Connect4Viewer(game=game)
view.initialize()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONUP and event.button == 1:
            if game.get_win() is None:
                game.place(pygame.mouse.get_pos()[0] // SQUARE_SIZE)
                print(game.get_board()[0][0])
                if game.get_win() is None:
                    turn = game.get_turn()
                    playMove(nw_c4,turn)
            else:
                game.reset_game()
                pygame.quit()

pygame.quit()

error: display Surface quit